In [1]:
import pandas as pd
import numpy as np
import os

In [60]:
# List only files in the current directory
files = [f for f in os.listdir('.') if os.path.isfile(f)]

for f in files:
    print(f)
    
txt_files = [f for f in files if f.lower().endswith('.txt')]

print(txt_files)

14th IIT Bombay Debate 2022 (23 rms).txt
14th IIT Bombay Debate 2022 (23 rms)_rounds_done.xlsx
14th IIT Bombay Debate 2022 (23 rms)_speakertab_done.xlsx
14th IIT Bombay Debate 2022 (23 rms)_teamtab_done.xlsx
15th IIT Bombay Debate 2023 (23 rms).txt
15th IIT Bombay Debate 2023 (23 rms)_rounds_done.xlsx
15th IIT Bombay Debate 2023 (23 rms)_speakertab_done.xlsx
15th IIT Bombay Debate 2023 (23 rms)_teamtab_done.xlsx
Africa IV 2022 (21 rms).txt
Africa IV 2022 (21 rms)_rounds_done.xlsx
Africa IV 2022 (21 rms)_speakertab_done.xlsx
Africa IV 2022 (21 rms)_teamtab_done.xlsx
All_Debates.xlsx
AODC BP 2022 (28 rms).txt
AODC BP 2022 (28 rms)_rounds_done.xlsx
AODC BP 2022 (28 rms)_speakertab_done.xlsx
AODC BP 2022 (28 rms)_teamtab_done.xlsx
Bali ABP 2022.txt
Bali ABP 2022_rounds_done.xlsx
Bali ABP 2022_speakertab_done.xlsx
Bali ABP 2022_teamtab_done.xlsx
Bates NAUDC 2023.txt
Bates NAUDC 2023_rounds_done.xlsx
Bates NAUDC 2023_speakertab_done.xlsx
Bates NAUDC 2023_teamtab_done.xlsx
Black Sea EUDC 2023

In [62]:
def get_comp_name(txt_file):
    comp_name = txt_file[:-4]
    print(comp_name)
    df = pd.read_csv(txt_file, delimiter ='\t', on_bad_lines='warn')
    print(len(df))
    return df, comp_name

for i in range(64,len(txt_files)):
    print(i)
    df, comp_name = get_comp_name(txt_files[i])
    
    
    
    stop_index = df[df['Rk'] == 'Rk'].index[0]

    teamtab = df.iloc[:stop_index]

    #Remove useless data
    target_column = 'Spk'
    column_index = teamtab.columns.get_loc(target_column)

    # Slice the DataFrame to keep columns up to and including the target column
    teamtab = teamtab.iloc[:, :column_index + 1]

    df2 = df.drop(teamtab.index)
    df2.reset_index(drop=True, inplace=True)

    stop_index2 = df2[df2['Rk'] == 'team'].index[0]
    speakertab = df2.iloc[:stop_index2]

    #remove top row & reset index:
    speakertab = speakertab.iloc[1:].reset_index(drop=True)

    #Remove useless data
    target_column = 'Spk'
    column_index = speakertab.columns.get_loc(target_column)

    # Slice the DataFrame to keep columns up to and including the target column
    speakertab = speakertab.iloc[:, :column_index + 1]

    speakertab = speakertab.rename(columns={'Rk': 'SpeakRank','team':'speaker','R1':'team'})

    # Get the first 4 and last column names
    first_columns = speakertab.columns[:4].tolist()
    last_column = speakertab.columns[-1]

    # Get the middle columns that need renaming
    #These will be the round results
    middle_columns = speakertab.columns[4:-1]

    # Create new names for the middle columns (e.g., 'NewCol1', 'NewCol2', ...)
    new_middle_columns = [f'R{i}' for i in range(1, len(middle_columns) + 1)]

    # Combine the first columns, renamed middle columns, and last column
    new_columns = first_columns + new_middle_columns + [last_column]

    # Apply the new column names to the DataFrame
    speakertab.columns = new_columns

    df3 = df2.drop(speakertab.index)
    df3 = df3.iloc[1:].reset_index(drop=True)
    df3.reset_index(drop=True, inplace=True)

    df3.loc[df3['R1'] == 'View Ballot', 'R1'] = df3['R2']
    df3 = df3.iloc[:,:4]
    df3["Round"] = 0

    round_counter = 0
    row_numbers = df3.index[df3["Rk"] == "team"].tolist()

    #Trick "dumbass": In order for the next for loop to iterate through, I append a 0 to the end of my list. 
    #This is needed, so when the loop is at the last round of the we don't end up with referring to a roundnumber, that would be 
    #Larger than the last element in the list. 
    #What I mean: check a few lines down
    row_numbers.append(0)
    print(row_numbers)


    df4 = df3
    for i in df4.index:
        if row_numbers[round_counter] == i:
            print(i)
            print(round_counter)

            #Trick "dumbass" --> in the last round if I did not append a 0, row_umbers[round_counter] would be
            #larger than the last element's position in the row_numbers list
            #But things just work perfectly this way, like a shitty spagetthi
            round_counter += 1
        df4.loc[i, "Round"] = round_counter  # Example logic
    row_numbers = row_numbers[:-1]

    df7 = df4
    df7['Speaker1'], df7['Speaker2'],df7['Speaker1_score'], df7['Speaker2_score'], df7['Comp_name'] = 0,0,0,0,comp_name
    for i in df7.index:
        #print(df7.Rk[i])
        loop_speaker1 = 'iron/swing'
        loop_speaker2 = 'iron/swing'
        loop_speaker1_score = 0
        loop_speaker2_score = 0
        loop_round = df7.Round[i]
        
        #At Cambridge IV 202
        speakertab.rename(columns=lambda c: 'team' if c == 'institution' else c, inplace=True)

        for j in speakertab.index:
            if df7.Rk[i] == speakertab.team[j]:
                if loop_speaker1 == 'iron/swing':
                    loop_speaker1 = speakertab.speaker[j]
                    if df7.team[i] == '1st' or df7.team[i] == '2nd' or df7.team[i] == '3rd' or df7.team[i] == '4th':
                        loop_speaker1_score = speakertab.iat[j,3+loop_round]
                    else:
                        loop_speaker1_score = 'outround'
                else:
                    loop_speaker2 = speakertab.speaker[j]
                    if df7.team[i] == '1st' or df7.team[i] == '2nd' or df7.team[i] == '3rd' or df7.team[i] == '4th':
                        loop_speaker2_score = speakertab.iat[j,3+loop_round]
                    else:
                        loop_speaker2_score = 'outround'
        df7.Speaker1[i] = loop_speaker1
        df7.Speaker2[i] = loop_speaker2
        df7.Speaker1_score[i] = loop_speaker1_score
        df7.Speaker2_score[i] = loop_speaker2_score

    df8 = df7
    df8

    grouped = df8.groupby(['Round', 'R1']).agg(list).reset_index()

    # Flatten the grouped DataFrame by splitting the lists into separate columns
    # Here, we use the `apply(pd.Series)` to convert each list into a separate column
    grouped_expanded = pd.concat([grouped[['Round', 'R1']], 
                                  grouped['Rk'].apply(pd.Series), 
                                  grouped['categories'].apply(pd.Series), 
                                  grouped['team'].apply(pd.Series),
                                  grouped['Speaker1'].apply(pd.Series),
                                  grouped['Speaker2'].apply(pd.Series),
                                  grouped['Speaker1_score'].apply(pd.Series),
                                  grouped['Speaker2_score'].apply(pd.Series)], axis=1)

    # Rename the columns to match the desired output

    grouped_expanded.columns = ['Round', 'Panel', 
                                'Team1', 'Team2', 'Team3', 'Team4',
                                'Team1 position', 'Team2 position', 'Team3 position', 'Team4 position',
                                'Team1 result', 'Team2 result', 'Team3 result', 'Team4 result',
                                'T1_S1 name', 'T2_S1 name', 'T3_S1 name', 'T4_S1 name',
                                'T1_S2 name', 'T2_S2 name', 'T3_S2 name', 'T4_S2 name',
                                'T1_S1 score', 'T2_S1 score', 'T3_S1 score', 'T4_S1 score',
                                'T1_S2 score', 'T2_S2 score', 'T3_S2 score', 'T4_S2 score']

    df9 = grouped_expanded


    df9 = df9.drop(df9[df9['Team1'] == 'team'].index)
    df9 = df9.reset_index()
    df9 = df9.drop('index', axis=1)

    def get_chair(text):
        return text.split("Ⓒ")[0]

    def get_wing1(paneltext):
        parts = paneltext.split(',')
        return parts[1] if len(parts) > 1 else None  # Returns second part if it exists

    def get_wing2(paneltext):
        parts = paneltext.split(',')
        if len(parts) > 2:
            value = parts[2]
            return value if 'Ⓣ' not in value else 0
        return 0

    def get_trainee1(paneltext):
        parts = paneltext.split(',')
        for part in parts:
            if 'Ⓣ' in part:
                return part
        return 0

    def get_trainee2(paneltext):
        parts = [part for part in paneltext.split(',') if 'Ⓣ' in part]
        return parts[1] if len(parts) > 1 else 0

    # Define column names
    columns = ['Round', 'Panel', 
               'OG', 'OO', 'CG','CO',
               'OG speaker 1', 'OG speaker 2', 'OO speaker 1','OO speaker 2',
               'CG speaker 1', 'CG speaker 2', 'CO speaker 1','CO speaker 2',
               'OG1 score', 'OG2 score', 'OO1 score','OO2 score',
               'CG1 score', 'CG2 score', 'CO1 score','CO2 score',
               'Roundtype', 'Panel_size', 'Voting_panel_size',
               'Chair', 'Wing1', 'Wing2',
               'Trainee1', 'Trainee2', 'Comp_name']

    # Get length of df9 
    rownumbers = df9.shape[0] 

    # Create an empty DataFrame with NaN (or pd.NA) values
    data = pd.DataFrame(data={col: [0] * rownumbers for col in columns})

    for i in df9.index:
        data.Round[i] = df9.Round[i]
        data.Panel[i] = df9.Panel[i]

        lineups = df9.iloc[i,6:10].tolist()
        pos_og = lineups.index('Opening Government')
        pos_oo = lineups.index('Opening Opposition')
        pos_cg = lineups.index('Closing Government')
        pos_co = lineups.index('Closing Opposition')
        teams_order = [pos_og, pos_oo, pos_cg, pos_co]

        data.OG[i] = df9.iat[i,teams_order[0]+2]
        data.OO[i] = df9.iat[i,teams_order[1]+2]
        data.CG[i] = df9.iat[i,teams_order[2]+2]
        data.CO[i] = df9.iat[i,teams_order[3]+2]

        data.iat[i,6] = df9.iat[i,teams_order[0]+14]
        data.iat[i,7] = df9.iat[i,teams_order[0]+18]
        data.iat[i,8] = df9.iat[i,teams_order[1]+14]
        data.iat[i,9] = df9.iat[i,teams_order[1]+18]
        data.iat[i,10] = df9.iat[i,teams_order[2]+14]
        data.iat[i,11] = df9.iat[i,teams_order[2]+18]
        data.iat[i,12] = df9.iat[i,teams_order[3]+14]
        data.iat[i,13] = df9.iat[i,teams_order[3]+18]

        if df9.iat[i,teams_order[0]+22] == 'outround':
            data.iat[i,14] = df9.iat[i,teams_order[0]+10]
            data.iat[i,15] = df9.iat[i,teams_order[0]+10]
            data.iat[i,16] = df9.iat[i,teams_order[1]+10]
            data.iat[i,17] = df9.iat[i,teams_order[1]+10]
            data.iat[i,18] = df9.iat[i,teams_order[2]+10]
            data.iat[i,19] = df9.iat[i,teams_order[2]+10]
            data.iat[i,20] = df9.iat[i,teams_order[3]+10]
            data.iat[i,21] = df9.iat[i,teams_order[3]+10]
            data.Roundtype[i] = "outround"

        else:
            data.iat[i,14] = df9.iat[i,teams_order[0]+22]
            data.iat[i,15] = df9.iat[i,teams_order[0]+26]
            data.iat[i,16] = df9.iat[i,teams_order[1]+22]
            data.iat[i,17] = df9.iat[i,teams_order[1]+26]
            data.iat[i,18] = df9.iat[i,teams_order[2]+22]
            data.iat[i,19] = df9.iat[i,teams_order[2]+26]
            data.iat[i,20] = df9.iat[i,teams_order[3]+22]
            data.iat[i,21] = df9.iat[i,teams_order[3]+26]
            data.Roundtype[i] = "inround"

        data.Panel_size[i] = df9.Panel[i].count(",")+1
        data.Voting_panel_size[i] = data.Panel_size[i] - df9.Panel[i].count("Ⓣ")
        if df9.Panel[i] != 0:
            data.Chair[i]  = get_chair(df9.Panel[i])

        if data.Panel_size[i] > 0:
            data.Wing1[i] = get_wing1(df9.Panel[i])
        if data.Panel_size[i] > 1:
            data.Wing2[i] = get_wing2(df9.Panel[i])
        if data.Panel_size[i] > 0:
            data.Trainee1[i] = get_trainee1(df9.Panel[i])
        if data.Panel_size[i] > 1:
            data.Trainee2[i] = get_trainee2(df9.Panel[i])

        data.Comp_name[i] = comp_name


    reordered_columns = ['Comp_name', 'Round', 'Chair', 'Roundtype',
               'OG', 'OO', 'CG','CO',
               'OG speaker 1', 'OG speaker 2', 'OO speaker 1','OO speaker 2',
               'CG speaker 1', 'CG speaker 2', 'CO speaker 1','CO speaker 2',
               'OG1 score', 'OG2 score', 'OO1 score','OO2 score',
               'CG1 score', 'CG2 score', 'CO1 score','CO2 score',
               'Panel_size', 'Voting_panel_size', 'Panel',
               'Wing1', 'Wing2', 'Trainee1', 'Trainee2']
    data = data[reordered_columns]


    teamtab.to_excel(comp_name+'_teamtab_done'+'.xlsx', index=True)
    speakertab.to_excel(comp_name+'_speakertab_done'+'.xlsx', index=True)
    data.to_excel(comp_name+'_rounds_done'+'.xlsx', index=True)
    

64
Vietnam WUDC 2024
3698
[0, 297, 594, 891, 1188, 1485, 1778, 2071, 2364, 2653, 2686, 2719, 2736, 2745, 2750, 2767, 2776, 2781, 2790, 0]
0
0
297
1
594
2
891
3
1188
4
1485
5
1778
6
2071
7
2364
8
2653
9
2686
10
2719
11
2736
12
2745
13
2750
14
2767
15
2776
16
2781
17
2790
18


C:\Users\andra\AppData\Local\Temp\ipykernel_31352\2008795280.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7.Speaker1[i] = loop_speaker1
C:\Users\andra\AppData\Local\Temp\ipykernel_31352\2008795280.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7.Speaker2[i] = loop_speaker2
C:\Users\andra\AppData\Local\Temp\ipykernel_31352\2008795280.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7.Speaker1_score[i] = loop_s

65
Warwick Pre-EUDC 2025 (29 rms)
1015
[0, 117, 234, 351, 468, 577, 594, 611, 620, 625, 634, 639, 0]
0
0
117
1
234
2
351
3
468
4
577
5
594
6
611
7
620
8
625
9
634
10
639
11


C:\Users\andra\AppData\Local\Temp\ipykernel_31352\2008795280.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7.Speaker1[i] = loop_speaker1
C:\Users\andra\AppData\Local\Temp\ipykernel_31352\2008795280.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7.Speaker2[i] = loop_speaker2
C:\Users\andra\AppData\Local\Temp\ipykernel_31352\2008795280.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7.Speaker1_score[i] = loop_s

66
Western IV 2022 (23 rms)
787
[0, 93, 186, 271, 352, 433, 450, 459, 464, 469, 0]
0
0
93
1
186
2
271
3
352
4
433
5
450
6
459
7
464
8
469
9


...

C:\Users\andra\AppData\Local\Temp\ipykernel_31352\2008795280.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7.Speaker1[i] = loop_speaker1
C:\Users\andra\AppData\Local\Temp\ipykernel_31352\2008795280.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7.Speaker2[i] = loop_speaker2
C:\Users\andra\AppData\Local\Temp\ipykernel_31352\2008795280.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7.Speaker1_score[i] = loop_s

69
Yale IV 2023 (25 rms)
856
[0, 101, 202, 303, 404, 501, 518, 527, 532, 541, 0]
0
0
101
1
202
2
303
3
404
4
501
5
518
6
527
7
532
8
541
9


C:\Users\andra\AppData\Local\Temp\ipykernel_31352\2008795280.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7.Speaker1[i] = loop_speaker1
C:\Users\andra\AppData\Local\Temp\ipykernel_31352\2008795280.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7.Speaker2[i] = loop_speaker2
C:\Users\andra\AppData\Local\Temp\ipykernel_31352\2008795280.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7.Speaker1_score[i] = loop_s

70
Zagreb EUDC 2022
1870
[0, 153, 306, 459, 612, 765, 918, 1067, 1216, 1365, 1374, 1379, 1396, 1405, 0]
0
0
153
1
306
2
459
3
612
4
765
5
918
6
1067
7
1216
8
1365
9
1374
10
1379
11
1396
12
1405
13


C:\Users\andra\AppData\Local\Temp\ipykernel_31352\2008795280.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7.Speaker1[i] = loop_speaker1
C:\Users\andra\AppData\Local\Temp\ipykernel_31352\2008795280.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7.Speaker2[i] = loop_speaker2
C:\Users\andra\AppData\Local\Temp\ipykernel_31352\2008795280.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7.Speaker1_score[i] = loop_s